## SQL Statement
This bit of code can be ran in psql to export the results of a query to a csv file in your postgres container.
After running it, copy it from your docker container into a local folder
docker cp container_id:/output.csv ~/Documents/output.csv

Run either sql statement but ensure to update compare_cols in the function to reflect the change. The first one will export data with a model_year column that shows the description from the model_year table ie 2020, 2021. The second one shows an id ie 1, 2, 3.

### -- if starting new use this line and update compare_csvs function below where it references model_year_id
\copy ( SELECT  icbc_registration_data.id AS registration_id, icbc_vehicle.id AS vehicle_id, model_name, make, model_year.description as model_year, icbc_registration_data.create_timestamp, icbc_registration_data.update_timestamp, vin::TEXT AS vin, icbc_registration_data.icbc_upload_date_id  FROM icbc_vehicle JOIN icbc_registration_data  ON icbc_vehicle.id = icbc_registration_data.icbc_vehicle_id JOIN model_year on model_year.id = model_year_id) TO 'output.csv' WITH CSV HEADER;


### -- old code where it just showed the model_year_id
\copy ( SELECT  icbc_registration_data.id AS registration_id, icbc_vehicle.id AS vehicle_id, model_name, make, model_year_id, icbc_registration_data.create_timestamp, icbc_registration_data.update_timestamp, vin::TEXT AS vin, icbc_registration_data.icbc_upload_date_id  FROM icbc_vehicle JOIN icbc_registration_data  ON icbc_vehicle.id = icbc_registration_data.icbc_vehicle_id ) TO 'output.csv' WITH CSV HEADER;


In [ ]:

import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill

def compare_csvs(file1, file2, output_file):
    # Read the CSVs into pandas DataFrames
    file_1_df = pd.read_csv(file1, dtype={'vin': str})
    file_2_df = pd.read_csv(file2, dtype={'vin': str})

    # Remove accidental quotes if they exist
    file_1_df["vin"] = file_1_df["vin"].str.strip('"') 
    file_2_df["vin"] = file_2_df["vin"].str.strip('"') 

  
    #if using the sql query that uses model_year instead of model_year_id use this
    compare_cols = ['vehicle_id', 'model_name', 'make', 'model_year']
    
    # otherwise use this
    # compare_cols = ['vehicle_id', 'model_name', 'make', 'model_year_id']

    # Merge the two DataFrames on the 'vin' column (the unique identifier) with an outer join
    merged_df = pd.merge(file_1_df, file_2_df, on="vin", how="outer", suffixes=('_file_1', '_file_2'))

    # Print the total number of rows before filtering
    print(f"Total rows before filtering: {len(merged_df)}")

    # Find VINs that exist in only one file
    vin_only_in_file_1 = merged_df[pd.isna(merged_df['registration_id_file_2'])]['vin']
    vin_only_in_file_2 = merged_df[pd.isna(merged_df['registration_id_file_1'])]['vin']
    
    # Print only unmatched VINs
    print("VINs only in File 1 (not in File 2):")
    print(vin_only_in_file_1.tolist())
    
    print("\nVINs only in File 2 (not in File 1):")
    print(vin_only_in_file_2.tolist())
    
    # Create a new DataFrame to keep track of differences
    for col in compare_cols:
        file_1_col = col + "_file_1"
        file_2_col = col + "_file_2"
        diff_col = col + "_diff"
        
        # Compare columns and mark differences
        merged_df[diff_col] = merged_df.apply(
            lambda row: 'yellow' if row[file_1_col] != row[file_2_col] else 'green', axis=1
        ).reset_index(drop=True) 

    # Filter out rows where there are no differences
    filtered_df = merged_df[
        (merged_df.filter(like='_diff') == 'yellow').any(axis=1)
    ]
    print(f"Total rows after filtering: {len(filtered_df)}")
    # Remove the '_diff' columns before saving the file
    filtered_df = filtered_df.drop(columns=[col + "_diff" for col in compare_cols])

    """ 
    Create excel and add headers with filenames
    """
    # write to an Excel file
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        filtered_df.to_excel(writer, index=False, sheet_name='Comparison')

    wb = openpyxl.load_workbook(output_file)
    sheet = wb['Comparison']

    file_1_name = file1.split('/')[-1]
    file_2_name = file2.split('/')[-1]

    sheet.insert_rows(1, amount=1)  # Insert row at the top

    # Merge cells for file names
    sheet.merge_cells(start_row=1, start_column=1, end_row=1, end_column=7)  # File 1 header
    sheet.merge_cells(start_row=1, start_column=9, end_row=1, end_column=sheet.max_column)  # File 2 header

    # Write file names in merged cells
    sheet.cell(row=1, column=1).value = f"File: {file_1_name}"
    sheet.cell(row=1, column=9).value = f"File: {file_2_name}"

    # Make headers bold
    bold_font = openpyxl.styles.Font(size=16, bold=True)
    sheet.cell(row=1, column=1).font = bold_font
    sheet.cell(row=1, column=9).font = bold_font

    """
    Add color formatting to sheet
    """
    # Get original row indices from merged_df before filtering
    filtered_indices = merged_df[
        (merged_df.filter(like='_diff') == 'yellow').any(axis=1)
    ].index.tolist()

    # Loop through filtered rows for applying highlighting
    for filtered_idx, row_idx in enumerate(filtered_indices):
        excel_row = filtered_idx + 3  # Excel rows start at 1, +3 for header rows
    
        # Highlight VIN column if it's missing in one file
        vin_col_idx = merged_df.columns.get_loc('vin') + 1
        vin_cell = sheet.cell(row=excel_row, column=vin_col_idx)
    
        # Check if VIN is unique to one of the files
        vin_is_unique = False
        if pd.isna(merged_df.loc[row_idx, 'registration_id_file_1']):  # Unique to file_2
            vin_cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Green
            vin_is_unique = True
        elif pd.isna(merged_df.loc[row_idx, 'registration_id_file_2']):  # Unique to file_1
            vin_cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Green
            vin_is_unique = True
    
        # Apply yellow highlighting for mismatched columns, **skip if VIN is unique**
        if not vin_is_unique:
            for col in compare_cols:
                file_1_col = col + "_file_1"
                file_2_col = col + "_file_2"
    
                # Ensure columns exist
                if file_1_col not in merged_df.columns or file_2_col not in merged_df.columns:
                    continue
                
                col_idx_1 = merged_df.columns.get_loc(file_1_col) + 1
                col_idx_2 = merged_df.columns.get_loc(file_2_col) + 1
    
                # Check if the values in these columns differ
                if merged_df.loc[row_idx, file_1_col] != merged_df.loc[row_idx, file_2_col]:
                    # Apply yellow highlighting for differences
                    sheet.cell(row=excel_row, column=col_idx_1).fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # Yellow
                    sheet.cell(row=excel_row, column=col_idx_2).fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # Yellow
    wb.save(output_file)
    print('Finished!')

In [ ]:
preprocessed_dataset = 'output-preprocessed.csv'
only_additions = 'output_only_additions.csv'
compare_csvs(preprocessed_dataset, only_additions, "only_additions.xlsx")

